In [ ]:
#installing fiftyone
!pip install fiftyone

In [ ]:
from ast import literal_eval
import pandas as pd
from tqdm.notebook import tqdm

TRAIN_PATH = '/kaggle/input/tensorflow-great-barrier-reef/train_images'
N_SAMP = 6000

df = pd.read_csv('/kaggle/input/tensorflow-great-barrier-reef/train.csv')

df['is_valid'] = df['video_id'] == 2
df['annotations'] = df['annotations'].apply(literal_eval)
df['path'] = df.apply(lambda row: f"{TRAIN_PATH}/video_{row['video_id']}/{row['video_frame']}.jpg", axis = 1)

df.tail()

Convert the Csv annotation to coco:

In [ ]:
def coco(df):
    
    annotion_id = 0
    images = []
    annotations = []

    categories = [{'id': 0, 'name': 'cots'}]

    for i, row in tqdm(df.iterrows(), total = len(df)):

        images.append({
            "id": i,
            "file_name": f"{row['image_id']}.jpg",
            "height": 720,
            "width": 1280,
        })
        for bbox in row['annotations']:
            annotations.append({
                "id": annotion_id,
                "image_id": i,
                "category_id": 0,
                "bbox": list(bbox.values()),
                "area": bbox['width'] * bbox['height'],
                "segmentation": [],
                "iscrowd": 0
            })
            annotion_id += 1

    json_file = {'categories':categories, 'images':images, 'annotations':annotations}
    return json_file

In [ ]:
json_train = coco(df[~df['is_valid']])
json_valid = coco(df[ df['is_valid']])

In [ ]:
#creating 2 json file and dumping train and valid file into it.
import json

with open('/kaggle/working/annotations_train.json', 'w', encoding='utf-8') as f:
    json.dump(json_train, f, ensure_ascii=True, indent=4)
    
with open('/kaggle/working/annotations_valid.json', 'w', encoding='utf-8') as f:
    json.dump(json_valid, f, ensure_ascii=True, indent=4)

In [ ]:
# Making 2 directories train2017 and test 2017
import os
os.makedirs('/kaggle/working/train2017', exist_ok=True)
os.makedirs('/kaggle/working/val2017', exist_ok=True)

In [ ]:
import shutil
for i, row in tqdm(df.iterrows(), total = len(df)):
    base_dir = 'val2017' if row['is_valid'] else 'train2017'
    fname = f"{row['image_id']}.jpg"
    shutil.copyfile(row['path'], f"/kaggle/working/{base_dir}/{fname}")

In [ ]:
# Load COCO formatted dataset
import fiftyone as fo
coco_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path= "/kaggle/working/train2017",
    labels_path="/kaggle/working/annotations_train.json",
    include_id=True
)

# Verify that the class list for our dataset was imported
print(coco_dataset.default_classes)  # ['airplane', 'apple', ...]

print(coco_dataset)

In [ ]:
fo.launch_app(coco_dataset)

In [ ]:
import fiftyone.brain as fob

fob.compute_uniqueness(coco_dataset)
rank_view = coco_dataset.sort_by("uniqueness")
    

In [ ]:
coco_dataset.default_classes